# Weather API from OpenWeather:

### Importing and installing 

In [3]:
!pip install pandas
import pandas as pd

In [4]:
import requests
import datetime

### Creating Dataframe with weather information using API

In [25]:
schema = "cities_info"
host = "127.0.0.1"
user = "root"
password = "Bar!ah123"
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [87]:
cities= pd.read_sql('SELECT City_id, City, Latitude, Longitude FROM cities', con= connection_string)

In [118]:
def weather_info(cities):
    
    # Initialize lists for storing data across multiple cities
    all_city_data_parameters = []

    temperature= []
    min_temp= []
    max_temp= []
    humidity= []
    wind_speed= []
    description= []
    date= []
    time_retrieved= []

    #API_key for weather api
    api_key= '4f595dfdc7bef7c7ccc1ea5359271f95'

    for index, row in cities.iterrows():
        Latitude = row['Latitude']
        Longitude = row['Longitude']
        City= row['City']
        
        weather_api= requests.get(f'https://api.openweathermap.org/data/2.5/forecast?lat={Latitude}&lon={Longitude}&appid={api_key}&units=metric')

        #Create json
        weather_json =  weather_api.json()

        for x in range(len(weather_json['list'])):
            #Temperature 
            temperature.append(weather_json['list'][x]['main']['temp'])

            #Min_Temperature & max_Temperature
            min_temp.append(weather_json['list'][x]['main']['temp_min'])
            max_temp.append(weather_json['list'][x]['main']['temp_max'])

            #Humidity 
            humidity.append(weather_json['list'][x]['main']['humidity'])

            #Wind Speed
            wind_speed.append(weather_json['list'][x]['wind']['speed'])

            #Description
            description.append(weather_json['list'][x]['weather'][0]['description'])

            #Time of record
            date.append(weather_json['list'][x]['dt_txt'])

            #Time when data retrieved
            time_retrieved.append(datetime.datetime.now().strftime('%Y/%m/%d, %H:%M'))

        #Creating DF for weather info
        Parameters= pd.DataFrame({'City': City,
                        'Date':date,
                        'Temperature': temperature,
                        'Minimum_Temperature': min_temp,
                        'Maximum_Temperature': max_temp,
                        'Humidity': humidity,
                        'Wind_Speed': wind_speed,
                        'Description': description,
                        'Time_Retrieved': time_retrieved}
      )
        #Append the current city's data to the all_city_data list
        all_city_data_parameters .append(Parameters)

    #Concatenate all the dataframes in the list to a single dataframe
    all_city_data_parameters = pd.concat(all_city_data_parameters, ignore_index=True)

    return (all_city_data_parameters)

#### Just a small check

In [119]:
weather_df= weather_info(cities)

In [120]:
weather_df['City'].unique()

array(['Berlin', 'Hamburg', 'Munich'], dtype=object)

In [121]:
len(weather_df)

240

In [122]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 240 non-null    object 
 1   Date                 240 non-null    object 
 2   Temperature          240 non-null    float64
 3   Minimum_Temperature  240 non-null    float64
 4   Maximum_Temperature  240 non-null    float64
 5   Humidity             240 non-null    int64  
 6   Wind_Speed           240 non-null    float64
 7   Description          240 non-null    object 
 8   Time_Retrieved       240 non-null    object 
dtypes: float64(4), int64(1), object(4)
memory usage: 17.0+ KB


In [123]:
weather_df['Date']= pd.to_datetime(weather_df['Date'])
weather_df['Time_Retrieved']= pd.to_datetime(weather_df['Time_Retrieved'])

In [124]:
weather_df.head()

,City,Date,Temperature,Minimum_Temperature,Maximum_Temperature,Humidity,Wind_Speed,Description,Time_Retrieved
0,Berlin,2025-03-20 21:00:00,8.14,8.14,8.14,61,2.23,broken clouds,2025-03-20 20:51:00
1,Berlin,2025-03-21 00:00:00,7.83,7.21,7.83,62,1.59,broken clouds,2025-03-20 20:51:00
2,Berlin,2025-03-21 03:00:00,6.65,5.91,6.65,66,1.97,broken clouds,2025-03-20 20:51:00
3,Berlin,2025-03-21 06:00:00,6.21,6.21,6.21,75,1.09,scattered clouds,2025-03-20 20:51:00
4,Berlin,2025-03-21 09:00:00,12.68,12.68,12.68,57,2.18,few clouds,2025-03-20 20:51:00


In [89]:
cities

,City_id,City,Latitude,Longitude
0,1,Berlin,53.0,13.0
1,2,Hamburg,54.0,10.0
2,3,Munich,48.0,12.0


### Merging two dataframes (cities & weather_df) to get weather_info with city_id rather than city and dropping columns we don't require

In [93]:
city_weather_info= pd.merge(weather_df, cities, how='left', on='City')

In [95]:
city_weather_info= city_weather_info.drop(['City', 'Latitude', 'Longitude'], axis= 1)

In [96]:
city_weather_info

,Date,Temperature,Minimum_Temperature,Maximum_Temperature,Humidity,Wind_Speed,Description,Time_Retrieved,City_id
0,2025-03-20 21:00:00,9.35,8.14,9.35,58,2.23,scattered clouds,"2025/03/20, 20:12",1
1,2025-03-21 00:00:00,8.12,7.21,8.12,62,1.59,broken clouds,"2025/03/20, 20:12",1
2,2025-03-21 03:00:00,5.91,5.91,5.91,68,1.97,scattered clouds,"2025/03/20, 20:12",1
3,2025-03-21 06:00:00,6.21,6.21,6.21,75,1.09,scattered clouds,"2025/03/20, 20:12",1
4,2025-03-21 09:00:00,12.68,12.68,12.68,57,2.18,few clouds,"2025/03/20, 20:12",1
...,...,...,...,...,...,...,...,...,...
235,2025-03-25 06:00:00,6.41,6.41,6.41,91,0.91,overcast clouds,"2025/03/20, 20:12",3
236,2025-03-25 09:00:00,9.46,9.46,9.46,78,0.49,light rain,"2025/03/20, 20:12",3
237,2025-03-25 12:00:00,12.78,12.78,12.78,62,0.83,light rain,"2025/03/20, 20:12",3
238,2025-03-25 15:00:00,10.53,10.53,10.53,78,1.84,light rain,"2025/03/20, 20:12",3


### Pushing info to sql database

In [108]:
#Transfer it to sql
city_weather_info.to_sql('weather',
                      if_exists = 'append',
                      con= connection_string,
                      index= False
)

240